# Seminar Algorithms from THE BOOK // TUM SS2022 // Alexander Feil

This notebook is supposed to be an addition to the Seminar report by Alexander Feil on the book "Algorithms from THE BOOK" written by Kenneth Lange. In this notebook, some "experiments" will be named. Those can be found at the end of the notebook. 

### 1.  Adjacency to neighbourhood

In this first code we will apply the adjacency to neighbourhood algorithm. The first code returns the neighbourhood vector for our matrix A and the second code returns the weight vector. Out of curiosity, this code also returns the duration of this code using the *@time* function. 

In [10]:
using BenchmarkTools
function adjacencytoneighbourhood(A::AbstractMatrix)
    (nodes,T) = (size(A,1), eltype(A)) 
    neighbour = [Vector{Int}() for i = 1:nodes]
    weight = [Vector{T}() for i=1:nodes]
        for i = 1:nodes
            for j = 1:nodes
                if A[i,j] != zero(T) 
                    push!(neighbour[i], j) 
                    push!(weight[i], A[i,j])
                end 
            end
        end
   return (neighbour, weight)
end

A₁=[0 3 6 0 0; 3 0 8 0 0; 6 8 0 0 0; 0 0 0 0 2; 0 0 0 2 0]
print(adjacencytoneighbourhood(A₁))
@benchmark adjacencytoneighbourhood(A₁)



([[2, 3], [1, 3], [1, 2], [5], [4]], [[3, 6], [3, 8], [6, 8], [2], [2]])

BenchmarkTools.Trial: 10000 samples with 9 evaluations.
 Range (min … max):  2.309 μs … 373.385 μs  ┊ GC (min … max): 0.00% … 98.72%
 Time  (median):     2.440 μs               ┊ GC (median):    0.00%
 Time  (mean ± σ):   2.929 μs ±   6.336 μs  ┊ GC (mean ± σ):  3.66% ±  1.71%

  ██▆▃▁▁                       ▁▁▁ ▁                          ▁
  █████████▇▇▇▇▆▆▆▆▅▆▆▆▆▇▇▇███████████▇█▆▆▅▄▅▃▅▄▄▄▃▃▃▅▅▄▄▄▃▄▅ █
  2.31 μs      Histogram: log(frequency) by time      8.11 μs <

 Memory estimate: 1.61 KiB, allocs estimate: 26.

### 2. Connected Components 
With this Algorithm we want to find the number of components in our graph and get a vector which notes the number of the component each node is in. 

In [3]:
function visit!(neighbour::Array{Array{Int, 1}, 1}, component::Vector{Int}, i::Int) 
    for j in neighbour[i] #goes through the neighbours of i
        if component[j] > 0 continue end #checks whether j has already been assigned to a component
        component[j] = component[i] #if it hasn't been assigned yet, the same component number of i is saved for j
        visit!(neighbour, component, j) #from j we then continue to further nodes. Once we have reached a leave we will go back stepwise until we have gone through the entire component. 
    end
end

function connect(neighbour::Array{Array{Int, 1}, 1}) #we require a nx1 vector with nx1 vectors as components. 
    nodes = length(neighbour) #set the total number of nodes we have to check
    component = zeros(Int, nodes) #we define our vector where the number of components will be noted. 0 Means it hasn't been checked yet. 
    componentcount = 0
    for i = 1:nodes #We want to check all nodes 
        if component[i] > 0 continue end #if v_i has been checked (i.e. greater 0), we dont't need to go over it again
        componentcount += 1 #If it hasn't been checked with visit! it must be in a different component. Hence we add one up
        component[i] = componentcount #we save the component number for the currently analysed node v_i
        visit!(neighbour, component, i) #We go through the neighbours of v_i and their neighbours until the component vi is in is covered. 
    end
    return (component, componentcount)
end




connect (generic function with 1 method)

In [4]:
A₂=[0   1   0   0   1   0   0   0   0   0;
    1   0   1   1   0   0   0   0   0   0;
    0   1   0   0   0   0   0   0   0   0;
    0   1   0   0   0   0   0   0   0   0;
    1   0   0   0   0   1   0   0   0   0;
    0   0   0   0   1   0   0   0   0   0;
    0   0   0   0   0   0   0   1   0   1;
    0   0   0   0   0   0   1   0   1   0;
    0   0   0   0   0   0   0   1   0   1;
    0   0   0   0   0   0   1   0   1   0]
neighboursofA₂ = adjacencytoneighbourhood(A₂)[1]


10-element Vector{Vector{Int64}}:
 [2, 5]
 [1, 3, 4]
 [2]
 [2]
 [1, 6]
 [5]
 [8, 10]
 [7, 9]
 [8, 10]
 [7, 9]

In [5]:
connect(neighboursofA₂)

([1, 1, 1, 1, 1, 1, 2, 2, 2, 2], 2)

### Dijkstra's Algorithm

In [6]:
using DataStructures #necessary for Priority Queue

function dijkstra(neighbour::Array{Array{Int, 1}, 1}, weight::Array{Array{T, 1},1}, source::Int) where T <: Int
    nodes = length(neighbour)
    node = collect(1:nodes)
    predecessor = zeros(Int, nodes)
    visited = falses(nodes)
    distance = fill(Inf, nodes) #single line for distance = zeros(nodes); fill!(distance, Inf)
    distance[source] = 0.0 #Set the distance for S to 0
    pq = PriorityQueue(zip(node, distance)) #See chapter 2 "Priority Queue"
    while !isempty(pq)
        (i,d) = peek(pq) #retrieve the node with the smallest remaining distance
        distance[i] = d #reset the distance
        visited[i] = true #note that it has been visited
        dequeue!(pq) #take it out of the PriorityQueue
        for k in 1:length(neighbour[i]) 
            j = neighbour[i][k] #going through the neighbours of i. We keep k bc v_j will be the kth entry in the weight vector. 
            if !visited[k] #if not visited
                dij = d + weight[i][k] #k needed here
                if pq[j] > dij
                    predecessor[j] = i #note what i's precedessor is in the path 
                    pq[j] = dij #adjust the current shortest path length to j. The order in pq adjusts itself automatically (see experiment 1)
                end
            end
        end
    end
    return (distance, predecessor)
end

dijkstra(adjacencytoneighbourhood(A)[1], adjacencytoneighbourhood(A)[2], 1)

LoadError: UndefVarError: A not defined

## Experiments

In [ ]:
#Does the queue reorder itself automatically?

function dijkstratest(neighbour::Array{Array{Int, 1}, 1}, weight::Array{Array{T, 1},1}, source::Int) where T <: Int
    nodes = length(neighbour)
    node = collect(1:nodes)
    predecessor = zeros(Int, nodes)
    visited = falses(nodes)
    distance = fill(Inf, nodes) #single line for distance = zeros(nodes); fill!(distance, Inf)
    distance[source] = 0.0 #Set the distance for S to 0
    pq = PriorityQueue(zip(node, distance)) #See chapter 2 "Priority Queue"
    for a in 1:2
        (i,d) = peek(pq) #retrieve the node with the smallest remaining distance
        distance[2] = 5 #reset the distance
        distance[3] = 3 #reset the distance
        pq[2] = 5
        pq[3]= 3
    end
    return (distance, predecessor, pq)
end

dijkstratest(adjacencytoneighbourhood(A)[1], adjacencytoneighbourhood(A)[2], 1)

([0.0, 5.0, 3.0, Inf, Inf], [0, 0, 0, 0, 0], PriorityQueue(1 => 0.0, 3 => 3.0, 2 => 5.0, 5 => Inf, 4 => Inf))